In [162]:
import numpy as np
import pandas as pd
import re
import time
import random

In [163]:
# referente a celula com:
# normalize textual columns #1 to #4 with transformers from huggingface
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from torch.nn import BCEWithLogitsLoss

In [164]:
# new libraries for splitting qualitative data and fine tune it
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score

In [165]:
with open('../../busca.txt', 'r') as f:
    busca = '../../' + f.read()
del f

## Análise e Tratamento inicial
dos dados

In [166]:
# Adiciona file_name ao Data_Frame a partir do índica
# Necessário para percorrer os arquivos txt
file_name_series = pd.read_csv('../../importador_frequenciado.csv', index_col='id').sort_index().file_name
df = pd.read_csv('../../exportador_frequenciado.csv', index_col='id').sort_index()
df['file_name'] = file_name_series
del file_name_series

In [167]:
pd.concat([df.head(10), df.tail(10)])

,lei,nº lei,data,assunto: emenda lei [f()] / é emendado [f(x)],outro assunto [o],assunto,n_palavras,freq_assunto,quais_f,quais_fx,...,função social especificamente ligada ao assunto,ramo (dogmático) do direito,ramo (sociológico) do direito,imposição/reconhecimento (pelo jurídico),ferramenta/ambiente,Artigo 1 revogado,efetivamente revogada [r],Unnamed: 27,Unnamed: 28,file_name
id,,,,,,,,,,,,,,,,,,,,,
1,L-12.015,12015,07/08/2009,f(),o,2,1799,"0,111%","('altera', 'l', 8069)",NaN,...,ECA,penal,violência;violência sexual;infância,reconhece,ambiente,NaN,NaN,NaN,NaN,l12015
2,L-12.965,12965,23/04/2014,f(x),NaN,128,6698,"1,911%",NaN,"('redação dada', 'l', 13709)",...,NaN,civil,comunicação,reconhece,ferramenta;ambiente,NaN,NaN,NaN,NaN,l12965
3,L-13.179,13179,22/10/2015,NaN,NaN,2,328,"0,610%",NaN,NaN,...,NaN,civil,econômico,impõe,ambiente,NaN,NaN,NaN,NaN,l13179
4,L-11.300,11300,10/05/2006,f(),o,4,1434,"0,279%","(None, 'l', 9504)",NaN,...,transparência,eleitoral,político,impõe,ferramenta,NaN,NaN,NaN,NaN,l11300
5,L-12.686,12686,18/07/2012,NaN,NaN,9,181,"4,972%",NaN,NaN,...,NaN,administrativo,nacional,impõe,ambiente,NaN,NaN,NaN,NaN,l12686
6,L-13.828,13828,13/05/2019,f(),NaN,3,229,"1,310%","('altera', 'l', 12485)",NaN,...,NaN,civil,econômico,impõe,ferramenta;ambiente,NaN,NaN,NaN,NaN,l13828
7,L-9.755,9755,16/12/1998,NaN,NaN,4,658,"0,608%",NaN,NaN,...,NaN,administrativo,burocrático,NaN,ferramenta,NaN,NaN,NaN,NaN,l9755
8,L-14.172,14172,10/06/2021,f(x),NaN,12,1903,"0,631%",NaN,"('redação dada', 'l', 14351);('instituído', 'l...",...,NaN,administrativo,educacional,impõe,ferramenta,NaN,NaN,NaN,NaN,l14172
9,L-11.341,11341,07/08/2006,f(),NaN,4,227,"1,762%","('altera', 'l', 5869)",NaN,...,NaN,processual,burocrático,reconhece,ambiente,NaN,r,NaN,NaN,l11341


In [168]:
df = df.loc[:,~df.columns.str.match(r'Unnamed')]
display(pd.concat([df.head(10), df.tail(10)]))

,lei,nº lei,data,assunto: emenda lei [f()] / é emendado [f(x)],outro assunto [o],assunto,n_palavras,freq_assunto,quais_f,quais_fx,...,benevolência,função social,função social especificamente ligada ao assunto,ramo (dogmático) do direito,ramo (sociológico) do direito,imposição/reconhecimento (pelo jurídico),ferramenta/ambiente,Artigo 1 revogado,efetivamente revogada [r],file_name
id,,,,,,,,,,,,,,,,,,,,,
1,L-12.015,12015,07/08/2009,f(),o,2,1799,"0,111%","('altera', 'l', 8069)",NaN,...,ameaça,segurança;segurança sexual;ECA,ECA,penal,violência;violência sexual;infância,reconhece,ambiente,NaN,NaN,l12015
2,L-12.965,12965,23/04/2014,f(x),NaN,128,6698,"1,911%",NaN,"('redação dada', 'l', 13709)",...,benefício,valoração;inclusão;consumo,NaN,civil,comunicação,reconhece,ferramenta;ambiente,NaN,NaN,l12965
3,L-13.179,13179,22/10/2015,NaN,NaN,2,328,"0,610%",NaN,NaN,...,benefício,cultura;consumo,NaN,civil,econômico,impõe,ambiente,NaN,NaN,l13179
4,L-11.300,11300,10/05/2006,f(),o,4,1434,"0,279%","(None, 'l', 9504)",NaN,...,benefício,democracia;transparência,transparência,eleitoral,político,impõe,ferramenta,NaN,NaN,l11300
5,L-12.686,12686,18/07/2012,NaN,NaN,9,181,"4,972%",NaN,NaN,...,benefício,inclusão;nacionalismo,NaN,administrativo,nacional,impõe,ambiente,NaN,NaN,l12686
6,L-13.828,13828,13/05/2019,f(),NaN,3,229,"1,310%","('altera', 'l', 12485)",NaN,...,ameaça,consumo,NaN,civil,econômico,impõe,ferramenta;ambiente,NaN,NaN,l13828
7,L-9.755,9755,16/12/1998,NaN,NaN,4,658,"0,608%",NaN,NaN,...,neutra,transparência,NaN,administrativo,burocrático,NaN,ferramenta,NaN,NaN,l9755
8,L-14.172,14172,10/06/2021,f(x),NaN,12,1903,"0,631%",NaN,"('redação dada', 'l', 14351);('instituído', 'l...",...,benefício,inclusão;educação,NaN,administrativo,educacional,impõe,ferramenta,NaN,NaN,l14172
9,L-11.341,11341,07/08/2006,f(),NaN,4,227,"1,762%","('altera', 'l', 5869)",NaN,...,benefício,burocrática,NaN,processual,burocrático,reconhece,ambiente,NaN,r,l11341


In [169]:
# Vamos substituir freq_assunto que está formatada em porcentagem por um float
# Após isso uma análise de value_counts para observar se há valores fora do padrão
# Ou mal escritos como "otológica" ao invés de "ontológica" e "normativo" ao invés de "normativa"
# Os exemplos acima já foram tratados na fonte e recarregados
df['freq_assunto'] = df['freq_assunto'].str.replace(',', '.').str.replace('%', '').astype(float) / 100
display(df.info())
print("_"*50)
for col in df.columns:
    print(col)
    display(df[col].value_counts())
    print("_"*5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 1 to 247
Data columns (total 27 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   lei                                              247 non-null    object 
 1   nº lei                                           247 non-null    int64  
 2   data                                             247 non-null    object 
 3   assunto: emenda lei [f()] / é emendado [f(x)]    170 non-null    object 
 4   outro assunto [o]                                204 non-null    object 
 5   assunto                                          247 non-null    int64  
 6   n_palavras                                       247 non-null    int64  
 7   freq_assunto                                     247 non-null    float64
 8   quais_f                                          124 non-null    object 
 9   quais_fx                        

None

__________________________________________________
lei


L-12.015    1
L-13.281    1
L-10.179    1
LCP-196     1
L-14.046    1
           ..
L-12.192    1
L-13.019    1
L-14.109    1
L-13.499    1
L-9.503     1
Name: lei, Length: 247, dtype: int64

_____
nº lei


...

_____
efetivamente revogada [r]


r       4
1/2r    1
Name: efetivamente revogada [r], dtype: int64

_____
file_name


l12015    1
l13281    1
l10179    1
lcp196    1
l14046    1
         ..
l12192    1
l13019    1
l14109    1
l13499    1
l10406    1
Name: file_name, Length: 247, dtype: int64

_____


#### Eliminação de colunas inúteis para análise

In [170]:
'''
 0   lei                                              247 non-null    object
 1   nº lei                                           247 non-null    int64
 2   data                                             247 non-null    object
 3   assunto: emenda lei [f()] / é emendado [f(x)]    170 non-null    object
 4   outro assunto [o]                                204 non-null    object
 5   assunto                                          247 non-null    int64
 6   n_palavras                                       247 non-null    int64
 7   freq_assunto                                     247 non-null    float64
 8   quais_f                                          124 non-null    object
 9   quais_fx                                         105 non-null    object
 10  quais_f_o                                        135 non-null    object
 11  quais_fx_o                                       141 non-null    object
 12  epígrafe                                         247 non-null    object
 13  ementa                                           247 non-null    object
 14  Artigo 1º                                        247 non-null    object
 15  URL                                              247 non-null    object
 16  dimensão                                         73 non-null     object
 17  benevolência                                     73 non-null     object
 18  função social                                    73 non-null     object
 19  função social especificamente ligada ao assunto  30 non-null     object
 20  ramo (dogmático) do direito                      73 non-null     object
 21  ramo (sociológico) do direito                    73 non-null     object
 22  imposição/reconhecimento (pelo jurídico)         68 non-null     object
 23  ferramenta/ambiente                              73 non-null     object
 24  Artigo 1 revogado                                8 non-null      object
 25  efetivamente revogada [r]                        5 non-null      object
 26  file_name                                        247 non-null    object
'''
# Colunas inúteis para remover 0, 1, 2, 5, 8, 9, 10, 11, 12, 15
# Ou seja: lei, nº lei, data, assunto, quais_f, quais_fx, quais_f_o, quais_fx_o, epígrafe, URL
df.rename(columns={'nº lei':'n_lei', 'assunto: emenda lei [f()] / é emendado [f(x)]':'assunto_f_ou_fx', 'outro assunto [o]':'outro_assunto', 'epígrafe':'epigrafe', 'Artigo 1º': 'artigo_1', 'URL':'url', 'dimensão':'dimensao', 'benevolência':'benevolencia', 'função social':'funcao_social', 'função social especificamente ligada ao assunto':'funcao_social_assunto', 'ramo (dogmático) do direito':'ramo_dogmatico', 'ramo (sociológico) do direito':'ramo_sociologico', 'imposição/reconhecimento (pelo jurídico)':'imposicao_reconhecimento', 'ferramenta/ambiente':'ferramenta_ambiente', 'Artigo 1 revogado':'artigo_1_revogado', 'efetivamente revogada [r]':'efetivamente_revogada'}, inplace=True)
df_old0 = df.copy()

df.drop(columns=['lei', 'n_lei', 'data', 'assunto', 'quais_f', 'quais_fx', 'quais_f_o', 'quais_fx_o', 'epigrafe', 'url'], inplace=True)
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 1 to 247
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   assunto_f_ou_fx           170 non-null    object 
 1   outro_assunto             204 non-null    object 
 2   n_palavras                247 non-null    int64  
 3   freq_assunto              247 non-null    float64
 4   ementa                    247 non-null    object 
 5   artigo_1                  247 non-null    object 
 6   dimensao                  73 non-null     object 
 7   benevolencia              73 non-null     object 
 8   funcao_social             73 non-null     object 
 9   funcao_social_assunto     30 non-null     object 
 10  ramo_dogmatico            73 non-null     object 
 11  ramo_sociologico          73 non-null     object 
 12  imposicao_reconhecimento  68 non-null     object 
 13  ferramenta_ambiente       73 non-null     object 
 14  artigo_1_r

None

##### Adicionar coluna "internacional"
além disso substituir já algumas colunas por 1 ou 0

In [171]:
regex_internacional = r'internacio|exporta|importa|passaporte|(?<!em língua )estrangeir|exterior'
# WRONG: df['internacional'] = 1 if df['Artigo 1º'].str.contains(regex_internacional, flags=re.IGNORECASE) | df['ementa'].str.contains(regex_internacional, flags=re.IGNORECASE) else 0
# ALSO WRONG: df['internacional'] = df[df['Artigo 1º'].str.contains(regex_internacional, flags=re.IGNORECASE) | df['ementa'].str.contains(regex_internacional, flags=re.IGNORECASE)]
for index, row in df.iterrows():
    if re.search(regex_internacional, row['artigo_1'], flags=re.IGNORECASE) or re.search(regex_internacional, row['ementa'], flags=re.IGNORECASE):
        df.at[index, 'internacional'] = 1
    else:
        df.at[index, 'internacional'] = 0

display(df.head(40))

,assunto_f_ou_fx,outro_assunto,n_palavras,freq_assunto,ementa,artigo_1,dimensao,benevolencia,funcao_social,funcao_social_assunto,ramo_dogmatico,ramo_sociologico,imposicao_reconhecimento,ferramenta_ambiente,artigo_1_revogado,efetivamente_revogada,file_name,internacional
id,,,,,,,,,,,,,,,,,,
1,f(),o,1799,0.00111,Altera o Título VI da Parte Especial do Decret...,Esta Lei altera o Título VI da Parte Especial ...,normativa,ameaça,segurança;segurança sexual;ECA,ECA,penal,violência;violência sexual;infância,reconhece,ambiente,NaN,NaN,l12015,0.0
2,f(x),NaN,6698,0.01911,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",ontológica,benefício,valoração;inclusão;consumo,NaN,civil,comunicação,reconhece,ferramenta;ambiente,NaN,NaN,l12965,0.0
3,NaN,NaN,328,0.00610,Obriga o fornecedor de ingresso para evento cu...,O fornecedor de ingresso para evento cultural ...,normativa,benefício,cultura;consumo,NaN,civil,econômico,impõe,ambiente,NaN,NaN,l13179,0.0
4,f(),o,1434,0.00279,"Dispõe sobre propaganda, financiamento e prest...","A Lei nº 9.504, de 30 de setembro de 1997, pas...",normativa,benefício,democracia;transparência,transparência,eleitoral,político,impõe,ferramenta,NaN,NaN,l11300,0.0
5,NaN,NaN,181,0.04972,Normatiza a divulgação de documentos instituci...,Esta Lei normatiza a divulgação de documentos ...,normativa,benefício,inclusão;nacionalismo,NaN,administrativo,nacional,impõe,ambiente,NaN,NaN,l12686,0.0
6,f(),NaN,229,0.01310,"Altera a Lei nº 12.485, de 12 de setembro de 2...","Esta Lei altera a Lei nº 12.485, de 12 de sete...",normativa,ameaça,consumo,NaN,civil,econômico,impõe,ferramenta;ambiente,NaN,NaN,l13828,0.0
7,NaN,NaN,658,0.00608,"Dispõe sobre a criação de ""homepage"" na ""Inter...",O Tribunal de Contas da União criará _homepage...,ontológica,neutra,transparência,NaN,administrativo,burocrático,NaN,ferramenta,NaN,NaN,l9755,0.0
8,f(x),NaN,1903,0.00631,"Dispõe sobre a garantia de acesso à internet, ...",Esta Lei dispõe sobre a assistência da União a...,normativa,benefício,inclusão;educação,NaN,administrativo,educacional,impõe,ferramenta,NaN,NaN,l14172,0.0
9,f(),NaN,227,0.01762,Altera o parágrafo único do art. 541 do Código...,"O parágrafo único do art. 541 da Lei nº 5.869,...",normativa,benefício,burocrática,NaN,processual,burocrático,reconhece,ambiente,NaN,r,l11341,0.0


In [172]:
df['internacional'].value_counts()

0.0    231
1.0     16
Name: internacional, dtype: int64

In [173]:
#já substituir colunas existentes por 1 ou 0 ou 0.5
for index, row in df.iterrows():
    list_f_or_fx = str(row['assunto_f_ou_fx']).split(';')
    halfador = False
    for item in list_f_or_fx:
        if not halfador:
            if item == '':
                halfador = True
            elif item == 'f()':
                df.at[index, 'f'] = 1
            elif item == 'f(x)':
                df.at[index, 'fx'] = 1
        else:
            if item == 'f()':
                df.at[index, 'f'] = 0.5
            elif item == 'f(x)':
                df.at[index, 'fx'] = 0.5

df.drop(columns=['assunto_f_ou_fx'], inplace=True)
df['f'] = df['f'].fillna(0)
df['fx'] = df['fx'].fillna(0)

df['outro_assunto'] = df['outro_assunto'].fillna(0).replace('o', 1).replace('o;?', 0.5)
df['artigo_1_revogado'] = df['artigo_1_revogado'].fillna(0).replace('r', 1).replace('1/2r', 0.5)
df['efetivamente_revogada'] = df['efetivamente_revogada'].fillna(0).replace('r', 1).replace('1/2r', 0.5)
display(df.head(40))

,outro_assunto,n_palavras,freq_assunto,ementa,artigo_1,dimensao,benevolencia,funcao_social,funcao_social_assunto,ramo_dogmatico,ramo_sociologico,imposicao_reconhecimento,ferramenta_ambiente,artigo_1_revogado,efetivamente_revogada,file_name,internacional,f,fx
id,,,,,,,,,,,,,,,,,,,
1,1.0,1799,0.00111,Altera o Título VI da Parte Especial do Decret...,Esta Lei altera o Título VI da Parte Especial ...,normativa,ameaça,segurança;segurança sexual;ECA,ECA,penal,violência;violência sexual;infância,reconhece,ambiente,0,0.0,l12015,0.0,1.0,0.0
2,0.0,6698,0.01911,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",ontológica,benefício,valoração;inclusão;consumo,NaN,civil,comunicação,reconhece,ferramenta;ambiente,0,0.0,l12965,0.0,0.0,1.0
3,0.0,328,0.00610,Obriga o fornecedor de ingresso para evento cu...,O fornecedor de ingresso para evento cultural ...,normativa,benefício,cultura;consumo,NaN,civil,econômico,impõe,ambiente,0,0.0,l13179,0.0,0.0,0.0
4,1.0,1434,0.00279,"Dispõe sobre propaganda, financiamento e prest...","A Lei nº 9.504, de 30 de setembro de 1997, pas...",normativa,benefício,democracia;transparência,transparência,eleitoral,político,impõe,ferramenta,0,0.0,l11300,0.0,1.0,0.0
5,0.0,181,0.04972,Normatiza a divulgação de documentos instituci...,Esta Lei normatiza a divulgação de documentos ...,normativa,benefício,inclusão;nacionalismo,NaN,administrativo,nacional,impõe,ambiente,0,0.0,l12686,0.0,0.0,0.0
6,0.0,229,0.01310,"Altera a Lei nº 12.485, de 12 de setembro de 2...","Esta Lei altera a Lei nº 12.485, de 12 de sete...",normativa,ameaça,consumo,NaN,civil,econômico,impõe,ferramenta;ambiente,0,0.0,l13828,0.0,1.0,0.0
7,0.0,658,0.00608,"Dispõe sobre a criação de ""homepage"" na ""Inter...",O Tribunal de Contas da União criará _homepage...,ontológica,neutra,transparência,NaN,administrativo,burocrático,NaN,ferramenta,0,0.0,l9755,0.0,0.0,0.0
8,0.0,1903,0.00631,"Dispõe sobre a garantia de acesso à internet, ...",Esta Lei dispõe sobre a assistência da União a...,normativa,benefício,inclusão;educação,NaN,administrativo,educacional,impõe,ferramenta,0,0.0,l14172,0.0,0.0,1.0
9,0.0,227,0.01762,Altera o parágrafo único do art. 541 do Código...,"O parágrafo único do art. 541 da Lei nº 5.869,...",normativa,benefício,burocrática,NaN,processual,burocrático,reconhece,ambiente,0,1.0,l11341,0.0,1.0,0.0


#### Multi-label classification
#### Antes de prosseguir para a exploração dos arquivos txt
Faremos uma conversão das colunas com diversos elementos separados por ';' para listas e com essas listas detectar todas as ocorrências únicas e criar colunas de categorias existentes.

In [174]:
# make a temporary dataframe with columns dimensao, benevolencia, funcao_social, funcao_social_assunto, ramo_dogmatico, ramo_sociologico, imposicao_reconhecimento, ferramenta_ambiente
df_temp = df[['dimensao', 'benevolencia', 'funcao_social', 'funcao_social_assunto', 'ramo_dogmatico', 'ramo_sociologico', 'imposicao_reconhecimento', 'ferramenta_ambiente']].copy()
for column in df_temp.columns:
    '''
    value_counted = df_temp[column].value_counts()
    # real_value_counted will be a set of unique values
    real_value_counted = set()
    # now we iterate over the value_counted Series
    for index, value in value_counted.items():
        if not re.search(r';', index):
            # if there is no ';' in the value, we add it to the set
            real_value_counted.add(index)
        else:
            # if there is a ';' in the value, we split it and iterate over the list
            for item in index.split(';'):
                # if the item is not empty, we add it to the set
                if item != '':
                    real_value_counted.add(item)
    # now we iterate over the set and create a new column for each item
    for item in real_value_counted:
        df_temp[column + '_' + item] = 0
    #'''
    # now we iterate over the rows of the temporary dataframe
    for index, row in df_temp.iterrows():
        # if the value of the row is not empty, we split it and iterate over the list
        if row[column] != '' and row[column] is not np.NaN:
            halfador = False
            if '?' in str(row[column]):
                halfador = True
            for item in str(row[column]).split(';'):
                # if the item is not empty, we set the value of the column to 1
                if item != '' and item != '?':
                    df_temp.at[index, column + '_' + item] = 1
                    if halfador:
                        df_temp.at[index, column + '_' + item] /= 2
    # now we drop the column
    df_temp.drop(columns=[column], inplace=True)
    df.drop(columns=[column], inplace=True)

# Sobre a benevolência neutra
Critérios:
Se foi identificado que a lei manifesta apenas particularidades técnicas do "como" tratar a internet/tecnologia, então é neutra.
Se foram identificadas tanto ameaças quanto benefícios, então é neutra.
Se foi identificado de forma controversa se a lei é ameaça ou benefício, por exemplo os casos 'ameaça;?' (ameaça para quem?), então é neutra.

In [175]:
# Create a mask where both 'benevolencia_benefício' and 'benevolencia_ameaça' are not NaN
mask = df_temp[['benevolencia_benefício', 'benevolencia_ameaça']].notna().all(axis=1)

# Set 'benevolencia_neutra' to 1 where 'benevolencia_benefício' and 'benevolencia_ameaça' are both 1
df_temp.loc[mask & ((df_temp['benevolencia_benefício'] == 1) & (df_temp['benevolencia_ameaça'] == 1)), 'benevolencia_neutra'] = 1.0

# Set 'benevolencia_neutra' to 0 where 'benevolencia_benefício' and 'benevolencia_ameaça' are both 0
df_temp.loc[mask & ((df_temp['benevolencia_benefício'] == 0) & (df_temp['benevolencia_ameaça'] == 0)), 'benevolencia_neutra'] = 0.0

# Set 'benevolencia_neutra' to 1.0 where 'benevolencia_benefício' or 'benevolencia_ameaça' is 0.5
df_temp.loc[mask & ((df_temp['benevolencia_benefício'] == 0.5) | (df_temp['benevolencia_ameaça'] == 0.5)), 'benevolencia_neutra'] = 1.0

df = pd.concat([df, df_temp], axis=1)

display(df.head(40))
del df_temp
del mask
# del real_value_counted
# del value_counted

,outro_assunto,n_palavras,freq_assunto,ementa,artigo_1,artigo_1_revogado,efetivamente_revogada,file_name,internacional,f,...,ramo_sociologico_saúde,ramo_sociologico_trabalho,ramo_sociologico_ética,ramo_sociologico_rural,ramo_sociologico_cultura,ramo_sociologico_esporte,imposicao_reconhecimento_reconhece,imposicao_reconhecimento_impõe,ferramenta_ambiente_ambiente,ferramenta_ambiente_ferramenta
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1799,0.00111,Altera o Título VI da Parte Especial do Decret...,Esta Lei altera o Título VI da Parte Especial ...,0,0.0,l12015,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
2,0.0,6698,0.01911,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",0,0.0,l12965,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0
3,0.0,328,0.00610,Obriga o fornecedor de ingresso para evento cu...,O fornecedor de ingresso para evento cultural ...,0,0.0,l13179,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
4,1.0,1434,0.00279,"Dispõe sobre propaganda, financiamento e prest...","A Lei nº 9.504, de 30 de setembro de 1997, pas...",0,0.0,l11300,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
5,0.0,181,0.04972,Normatiza a divulgação de documentos instituci...,Esta Lei normatiza a divulgação de documentos ...,0,0.0,l12686,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
6,0.0,229,0.01310,"Altera a Lei nº 12.485, de 12 de setembro de 2...","Esta Lei altera a Lei nº 12.485, de 12 de sete...",0,0.0,l13828,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
7,0.0,658,0.00608,"Dispõe sobre a criação de ""homepage"" na ""Inter...",O Tribunal de Contas da União criará _homepage...,0,0.0,l9755,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,0.0,1903,0.00631,"Dispõe sobre a garantia de acesso à internet, ...",Esta Lei dispõe sobre a assistência da União a...,0,0.0,l14172,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
9,0.0,227,0.01762,Altera o parágrafo único do art. 541 do Código...,"O parágrafo único do art. 541 da Lei nº 5.869,...",0,1.0,l11341,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN


In [176]:
display(df.describe())

,outro_assunto,n_palavras,freq_assunto,artigo_1_revogado,efetivamente_revogada,internacional,f,fx,dimensao_normativa,dimensao_ontológica,...,ramo_sociologico_saúde,ramo_sociologico_trabalho,ramo_sociologico_ética,ramo_sociologico_rural,ramo_sociologico_cultura,ramo_sociologico_esporte,imposicao_reconhecimento_reconhece,imposicao_reconhecimento_impõe,ferramenta_ambiente_ambiente,ferramenta_ambiente_ferramenta
count,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,59.0,23.0,...,3.0,3.0,3.0,1.0,1.0,1.0,35.0,35.0,34.0,51.0
mean,0.821862,12433.400810,0.004009,0.032389,0.018219,0.064777,0.497976,0.425101,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.380746,19845.630897,0.006443,0.177390,0.130166,0.246632,0.498979,0.495362,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
min,0.000000,181.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.000000,1901.500000,0.000800,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.000000,5429.000000,0.001660,0.000000,0.000000,0.000000,0.500000,0.000000,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.000000,15188.000000,0.003985,0.000000,0.000000,0.000000,1.000000,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.000000,217821.000000,0.049720,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [177]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 1 to 247
Data columns (total 92 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   outro_assunto                                247 non-null    float64
 1   n_palavras                                   247 non-null    int64  
 2   freq_assunto                                 247 non-null    float64
 3   ementa                                       247 non-null    object 
 4   artigo_1                                     247 non-null    object 
 5   artigo_1_revogado                            247 non-null    int64  
 6   efetivamente_revogada                        247 non-null    float64
 7   file_name                                    247 non-null    object 
 8   internacional                                247 non-null    float64
 9   f                                            247 non-null    float64
 10  fx

None

In [178]:
regex = r'(?<=\n\n)((.|\n.)+(internet|on-line|online|tecnol|digital|computador|computaciona|eletron|eletrôn|não presencial|virtual|virtuais|banda.larga| redes?\s|veículo de comunicação social)(.|\n.)+(?=\n\n))'

In [179]:
def preprocess_text(text):
    # Replace all newline characters with a space
    text = text.replace('\n', ' ')

    # Remove all words ending in a dot followed by a space
    text = re.sub(r'\b\w+\.\s', '', text)

    # Remove all special characters
    text = re.sub(r'[^\w\s]', '', text)

    # Remove all numbers
    text = re.sub(r'\d+', '', text)

    # Remove all words with less than 3 letters
    text = re.sub(r'\b\w{1,2}\b', '', text)

    # Remove all duplicate spaces
    text = re.sub(r'\s+', ' ', text)

    return text.lower()

In [180]:
def lists_of_law_interaction(file_name):
    with open(busca + "/txt_laws/" + file_name + ".txt", 'r') as f:
        f_r = f.read()

        table_found = re.search(r'\|\s+\|\s+\|\s+[\|\s]*', 'r')

        if table_found:
            f_r = f_r[:table_found.start()]
            # the line above is doing the following:
            # if there is a table, it will cut the text before the table
            # and by cut I mean it will only consider the text before the table
        all_groups1 = [groups[0] for groups in re.findall(regex, f_r)]
        # above is almost correct, but I don't want to append to my list if the string starts with ~~
        all_occ_curr = [preprocess_text(group) for group in all_groups1 if not group.startswith('~~')]
        all_occ_ncurr = [preprocess_text(group) for group in all_groups1 if group.startswith('~~')]
    return bool(table_found), all_occ_curr, all_occ_ncurr

In [181]:
def create_table_found_and_ocorrencias(row):
    file_name = row.file_name
    table_found, all_occ_curr, all_occ_ncurr = lists_of_law_interaction(file_name)
    row['table_found'] = 1 if table_found else 0
    row['all_occ_curr'] = all_occ_curr
    row['all_occ_ncurr'] = all_occ_ncurr
    return row

##### Temporary action

In [182]:
# for now I'll drop efetivamente_revogada column because to analyze it I should go in depth in other laws my law changes
df.drop(['efetivamente_revogada'], axis=1, inplace=True)

In [183]:
df = df.apply(create_table_found_and_ocorrencias, axis=1)
df.drop(columns=['file_name'], inplace=True)
df.head()

,outro_assunto,n_palavras,freq_assunto,ementa,artigo_1,artigo_1_revogado,internacional,f,fx,dimensao_normativa,...,ramo_sociologico_rural,ramo_sociologico_cultura,ramo_sociologico_esporte,imposicao_reconhecimento_reconhece,imposicao_reconhecimento_impõe,ferramenta_ambiente_ambiente,ferramenta_ambiente_ferramenta,table_found,all_occ_curr,all_occ_ncurr
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1799,0.00111,Altera o Título VI da Parte Especial do Decret...,Esta Lei altera o Título VI da Parte Especial ...,0,0.0,1.0,0.0,1.0,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,0,[ _o_ incorre nas penas previstas caput deste ...,[]
2,0.0,6698,0.01911,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",0,0.0,0.0,1.0,NaN,...,NaN,NaN,NaN,1.0,NaN,1.0,1.0,0,[ esta lei estabelece princípios garantias dir...,[vii aplicações internet conjunto funcionalida...
3,0.0,328,0.00610,Obriga o fornecedor de ingresso para evento cu...,O fornecedor de ingresso para evento cultural ...,0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0,[obriga fornecedor ingresso para evento cultur...,[]
4,1.0,1434,0.00279,"Dispõe sobre propaganda, financiamento e prest...","A Lei nº 9.504, de 30 de setembro de 1997, pas...",0,0.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,1.0,0,[ cheques cruzados nominais transferência elet...,[]
5,0.0,181,0.04972,Normatiza a divulgação de documentos instituci...,Esta Lei normatiza a divulgação de documentos ...,0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0,[normatiza divulgação documentos institucionai...,[]


In [184]:
df_old1 = df.copy()

# Initialize a list to hold the new rows
new_rows = []

for index, row in df.iterrows():
    if len(row['all_occ_curr']) == 0 and len(row['all_occ_ncurr']) == 0:
        # Exclude 'all_occ_curr' and 'all_occ_ncurr' from the row dictionary
        # handle the order of the columns and append what is left
        row_dict = {'original_id': index, 'ementa': row['ementa'], 'artigo_1': row['artigo_1'],
                    'ocorrencia': np.nan, 'ocorrencia_revogada': np.nan, 'table_found': row['table_found'],
                    **{col: row[col] for col in df.columns if col not in ['ementa', 'artigo_1', 'all_occ_curr', 'all_occ_ncurr']}}
        new_rows.append(row_dict)
    else:
        for occ in row['all_occ_curr']:
            # Exclude 'all_occ_curr' and 'all_occ_ncurr' from the row dictionary
            # handle the order of the columns and append what is left
            row_dict = {'original_id': index, 'ementa': row['ementa'], 'artigo_1': row['artigo_1'],
                        'ocorrencia': occ, 'ocorrencia_revogada': np.nan, 'table_found': row['table_found'],
                        **{col: row[col] for col in df.columns if col not in ['ementa', 'artigo_1', 'all_occ_curr', 'all_occ_ncurr']}}
            new_rows.append(row_dict)

        for nocc in row['all_occ_ncurr']:
            # Exclude 'all_occ_curr' and 'all_occ_ncurr' from the row dictionary
            # handle the order of the columns and append what is left
            row_dict = {'original_id': index, 'ementa': row['ementa'], 'artigo_1': row['artigo_1'],
                        'ocorrencia': np.nan, 'ocorrencia_revogada': nocc, 'table_found': row['table_found'],
                        **{col: row[col] for col in df.columns if col not in ['ementa', 'artigo_1', 'all_occ_curr', 'all_occ_ncurr']}}
            new_rows.append(row_dict)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(new_rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3383 entries, 0 to 3382
Data columns (total 94 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   original_id                                  3383 non-null   int64  
 1   ementa                                       3383 non-null   object 
 2   artigo_1                                     3383 non-null   object 
 3   ocorrencia                                   3095 non-null   object 
 4   ocorrencia_revogada                          276 non-null    object 
 5   table_found                                  3383 non-null   int64  
 6   outro_assunto                                3383 non-null   float64
 7   n_palavras                                   3383 non-null   int64  
 8   freq_assunto                                 3383 non-null   float64
 9   artigo_1_revogado                            3383 non-null   int64  
 10  

In [185]:
# Select columns after the #12 column and check if all values are NaN
is_all_nan = df.iloc[:, 13:].isna().all(axis=1)
display(is_all_nan.head())
display(is_all_nan.tail())


# Replace NaN values with 0 in rows where not all values are NaN
df.loc[~is_all_nan, df.columns[13:]] = df.loc[~is_all_nan, df.columns[13:]].fillna(0)
display(df.head())
display(df.tail())

del is_all_nan

0    False
1    False
2    False
3    False
4    False
dtype: bool

3378    True
3379    True
3380    True
3381    True
3382    True
dtype: bool

,original_id,ementa,artigo_1,ocorrencia,ocorrencia_revogada,table_found,outro_assunto,n_palavras,freq_assunto,artigo_1_revogado,...,ramo_sociologico_saúde,ramo_sociologico_trabalho,ramo_sociologico_ética,ramo_sociologico_rural,ramo_sociologico_cultura,ramo_sociologico_esporte,imposicao_reconhecimento_reconhece,imposicao_reconhecimento_impõe,ferramenta_ambiente_ambiente,ferramenta_ambiente_ferramenta
0,1,Altera o Título VI da Parte Especial do Decret...,Esta Lei altera o Título VI da Parte Especial ...,_o_ incorre nas penas previstas caput deste a...,NaN,0,1.0,1799,0.00111,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",esta lei estabelece princípios garantias dire...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem como funda...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem seguintes ...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",viii liberdade dos modelos negócios promovidos...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


,original_id,ementa,artigo_1,ocorrencia,ocorrencia_revogada,table_found,outro_assunto,n_palavras,freq_assunto,artigo_1_revogado,...,ramo_sociologico_saúde,ramo_sociologico_trabalho,ramo_sociologico_ética,ramo_sociologico_rural,ramo_sociologico_cultura,ramo_sociologico_esporte,imposicao_reconhecimento_reconhece,imposicao_reconhecimento_impõe,ferramenta_ambiente_ambiente,ferramenta_ambiente_ferramenta
3378,245,Aprova a Consolidação das Leis do Trabalho.,Fica aprovada a Consolidação das Leis do Traba...,NaN,valor multa será reduzido trinta por cento in...,0,1.0,217821,0.00057,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3379,245,Aprova a Consolidação das Leis do Trabalho.,Fica aprovada a Consolidação das Leis do Traba...,NaN,valor multa será reduzido cinquenta por cento...,0,1.0,217821,0.00057,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3380,246,Institui o Código de Processo Civil.,"A jurisdição civil, contenciosa e voluntária, ...",NaN,NaN,0,1.0,281,0.00000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3381,247,Institui o Código Civil.,Toda pessoa é capaz de direitos e deveres na o...,NaN,pessoas jurídicas direito privado sem prejuíz...,0,1.0,4026,0.00050,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3382,247,Institui o Código Civil.,Toda pessoa é capaz de direitos e deveres na o...,NaN,pessoas jurídicas direito privado sem prejuíz...,0,1.0,4026,0.00050,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3383 entries, 0 to 3382
Data columns (total 94 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   original_id                                  3383 non-null   int64  
 1   ementa                                       3383 non-null   object 
 2   artigo_1                                     3383 non-null   object 
 3   ocorrencia                                   3095 non-null   object 
 4   ocorrencia_revogada                          276 non-null    object 
 5   table_found                                  3383 non-null   int64  
 6   outro_assunto                                3383 non-null   float64
 7   n_palavras                                   3383 non-null   int64  
 8   freq_assunto                                 3383 non-null   float64
 9   artigo_1_revogado                            3383 non-null   int64  
 10  

In [187]:
df.head()

,original_id,ementa,artigo_1,ocorrencia,ocorrencia_revogada,table_found,outro_assunto,n_palavras,freq_assunto,artigo_1_revogado,...,ramo_sociologico_saúde,ramo_sociologico_trabalho,ramo_sociologico_ética,ramo_sociologico_rural,ramo_sociologico_cultura,ramo_sociologico_esporte,imposicao_reconhecimento_reconhece,imposicao_reconhecimento_impõe,ferramenta_ambiente_ambiente,ferramenta_ambiente_ferramenta
0,1,Altera o Título VI da Parte Especial do Decret...,Esta Lei altera o Título VI da Parte Especial ...,_o_ incorre nas penas previstas caput deste a...,NaN,0,1.0,1799,0.00111,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",esta lei estabelece princípios garantias dire...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem como funda...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem seguintes ...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",viii liberdade dos modelos negócios promovidos...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [188]:
df_all = df.copy()

# keep only the rows for which the column #13 is not NaN
df = df[~df.iloc[:, 13].isna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1035 entries, 0 to 2916
Data columns (total 94 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   original_id                                  1035 non-null   int64  
 1   ementa                                       1035 non-null   object 
 2   artigo_1                                     1035 non-null   object 
 3   ocorrencia                                   962 non-null    object 
 4   ocorrencia_revogada                          68 non-null     object 
 5   table_found                                  1035 non-null   int64  
 6   outro_assunto                                1035 non-null   float64
 7   n_palavras                                   1035 non-null   int64  
 8   freq_assunto                                 1035 non-null   float64
 9   artigo_1_revogado                            1035 non-null   int64  
 10  

# Very proud of it
## Data treated to a wide extent
##### until now most things in this model are repeatable for many other sociological researches on the Brazilian Law
with a few tweaks here and there because categories would change

In [189]:
# normalizethe column 'n_palavras' in the following way df_norm = (df - df.min()) / (df.max() - df.min())
df['n_palavras'] = (df['n_palavras'] - df['n_palavras'].min()) / (df['n_palavras'].max() - df['n_palavras'].min())

In [190]:
display(df.n_palavras)

0       0.040043
1       0.161284
2       0.161284
3       0.161284
4       0.161284
          ...   
2912    0.995966
2913    0.995966
2914    0.995966
2915    0.995966
2916    0.995966
Name: n_palavras, Length: 1035, dtype: float64

# GPU TIME!

In [191]:
# check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1050 Ti


In [192]:
'''
# normalize textual columns #1 to #4 with transformers from huggingface
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model_quali_old = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=81)

# Define the optimizer
optimizer = AdamW(model_quali_old.parameters(), lr=1e-5)

model_quali_old.train()  # Set the model to training mode

for row in df.itertuples():
    # concatenate the text
    # row.ocorrencia and row.ocorrencia_revogada are exclusive, text fragment "revogada-ou-alterada" will be present if ocorrencia_revogada is not NaN and "vigente" will be present if ocorrencia is not NaN
    if row.ocorrencia_revogada is not np.nan:
        text1 = f"{preprocess_text(row.ementa)} {preprocess_text(row.artigo_1)}"
        text2 = f"R: {row.ocorrencia_revogada}"
    else:
        text1 = f"{preprocess_text(row.ementa)} {preprocess_text(row.artigo_1)}"
        text2 = f"V: {row.ocorrencia}"

    # tokenize the text
    tokens1 = tokenizer(text1, padding=True, truncation=True, max_length=196, return_tensors="pt")
    tokens2 = tokenizer(text2, padding=True, truncation=True, max_length=316, return_tensors="pt")
    tokens = {**tokens1, **tokens2}

    # get the model's labels
    labels = torch.tensor(list(row)[14:]).float()  # ensure it's float

    # Forward pass
    outputs = model_quali_old(**tokens)

    # get the logits
    logits = outputs.logits

    # Compute loss
    criterion = BCEWithLogitsLoss()
    loss = criterion(logits, labels.unsqueeze(0))  # add an extra dimension to labels to match logits' shape

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()

    # Clear gradients
    optimizer.zero_grad()

#'''

'\n# normalize textual columns #1 to #4 with transformers from huggingface\ntokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")\nmodel_quali_old = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=81)\n\n# Define the optimizer\noptimizer = AdamW(model_quali_old.parameters(), lr=1e-5)\n\nmodel_quali_old.train()  # Set the model to training mode\n\nfor row in df.itertuples():\n    # concatenate the text\n    # row.ocorrencia and row.ocorrencia_revogada are exclusive, text fragment "revogada-ou-alterada" will be present if ocorrencia_revogada is not NaN and "vigente" will be present if ocorrencia is not NaN\n    if row.ocorrencia_revogada is not np.nan:\n        text1 = f"{preprocess_text(row.ementa)} {preprocess_text(row.artigo_1)}"\n        text2 = f"R: {row.ocorrencia_revogada}"\n    else:\n        text1 = f"{preprocess_text(row.ementa)} {preprocess_text(row.artigo_1)}"\n        text2 = f"V: {row.ocorre

In [193]:
# model_quali_old.save_pretrained("../../model_quali_old")

In [194]:
# load the model
# model_quali_old = AutoModelForSequenceClassification.from_pretrained("../../model_quali_old", num_labels=81)

## Now with fine tuning and batch training

In [195]:
df.iloc[:, 1:5].iloc[0][3]

nan

In [196]:
# row = df.iloc[0][8]
# display(row)
# del row

In [197]:
# for row in df.itertuples():
#     print(list(row)[9])
#     break

In [198]:
# in itertuples there's an useless index in position 0, that makes it uses position 14 instead of 13

In [199]:
# '''
# normalize textual columns #1 to #4 with transformers from huggingface
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model_quali = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=81)

# Define the optimizer
optimizer = AdamW(model_quali.parameters(), lr=1e-5)

# Define criterion to compute loss
criterion = BCEWithLogitsLoss()

# Create a Dataset
class LeiQualiDataset(torch.utils.data.Dataset):
    def __init__(self, xy,  tokenizer):
        self.xy = xy
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        xy_row = self.xy.iloc[index]
        # concatenate the text
        # row.ocorrencia and row.ocorrencia_revogada are exclusive, text fragment "revogada-ou-alterada" will be present if ocorrencia_revogada is not NaN and "vigente" will be present if ocorrencia is not NaN
        if row[3] is not np.nan:
            text1 = f"{preprocess_text(xy_row[0])} {preprocess_text(xy_row[1])}"
            text2 = f"R: {xy_row[3]}"
        else:
            text1 = f"{preprocess_text(xy_row[0])} {preprocess_text(xy_row[1])}"
            text2 = f"V: {xy_row[2]}"

        # tokenize the text
        tokens1 = self.tokenizer(text1, padding='max_length', truncation=True, max_length=196, return_tensors="pt")
        tokens2 = self.tokenizer(text2, padding='max_length', truncation=True, max_length=316, return_tensors="pt")

        # Create new tensors by concatenating the input_ids, attention_masks and token_type_ids
        # and padding the first part to the maximum length of 512 tokens
        input_ids = torch.cat([tokens1['input_ids'].squeeze(), tokens2['input_ids'].squeeze()], dim=-1)
        attention_mask = torch.cat([tokens1['attention_mask'].squeeze(), tokens2['attention_mask'].squeeze()], dim=-1)

        # Won't happen, I just have separated sentences, not question answers or anything like that
        # # If the tokenizer returns 'token_type_ids', handle them as well
        # if 'token_type_ids' in tokens1 and 'token_type_ids' in tokens2:
        #     token_type_ids = torch.cat([tokens1['token_type_ids'], tokens2['token_type_ids']], dim=-1)
        #     tokens = {'input_ids': input_ids, 'attention_mask': attention_mask, 'token_type_ids': token_type_ids}
        # else:
        #     tokens = {'input_ids': input_ids, 'attention_mask': attention_mask}
        tokens = {'input_ids': input_ids, 'attention_mask': attention_mask}

        # get the model's labels
        labels = torch.tensor(xy_row[4:].values.astype(float)).float()  # ensure it's float
        return tokens, labels

    def __len__(self):
        return len(self.xy)

#'''

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [200]:
# # X will have only columns 1, 2, 3 and 4, which are ementa, artigo_1, ocorrencia and ocorrencia_revogada
# # y will have the row.iloc[13:] equivalent, which are the labels
# random_state = 409
# X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:5], df.iloc[:, 13:], test_size=0.5, random_state=random_state)

In [201]:
# pd.concat([X_test, y_test], axis=1).iloc[0][4:].values.astype(float)

In [202]:
# # Create DataLoader for training and testing (validation)
# train_dataset = LeiQualiDataset(pd.concat([X_train, y_train], axis=1), tokenizer)
# test_dataset = LeiQualiDataset(pd.concat([X_test, y_test], axis=1), tokenizer)
# train_dataloader = DataLoader(train_dataset, batch_size=7, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=7)

In [203]:
# for epoch in range(25):
#     # Training
#     model_quali.train()
#     for batch in train_dataloader:
#         # Get data
#         tokens, labels = batch
# 
#         print(tokens)
#         break
#     break

In [204]:
'''
last_average_f1_score = 0
epoch_range = 6
start_time = time.time()
epoch_broken = 0
random_state = [0] * epoch_range
for hyper_epoch in range(3):
    if epoch_broken > 2: break
    print(f'______________________\nHyper epoch {hyper_epoch}\n______________________\n')
    for epoch in range(epoch_range):
        elapsed_time = time.time() - start_time
        if hyper_epoch == 0:
            random.seed(elapsed_time)
            # Generate a new random state for each epoch
            random_state[epoch] = random.randint(0, 409)
        # the dataframe will be sliced in epoch_range parts, each epoch will use 1/epoch_range of the dataframe
        df_slice = df[df.index % epoch_range == epoch]
        # X will have only columns 1, 2, 3 and 4, which are ementa, artigo_1, ocorrencia and ocorrencia_revogada
        # y will have the row.iloc[13:] equivalent, which are the labels
        X_train, X_test, y_train, y_test = train_test_split(df_slice.iloc[:, 1:5], df_slice.iloc[:, 13:], test_size=0.25, random_state=random_state[epoch])
        # Recreate DataLoader for training and testing (validation)
        train_dataset = LeiQualiDataset(pd.concat([X_train, y_train], axis=1), tokenizer)
        test_dataset = LeiQualiDataset(pd.concat([X_test, y_test], axis=1), tokenizer)
        train_dataloader = DataLoader(train_dataset, batch_size=7, shuffle=True)
        test_dataloader = DataLoader(test_dataset, batch_size=7)

        print(f"-- on hyper epoch {hyper_epoch} and using random state {random_state[epoch]} --")
        print(f"Epoch {epoch + 1} of {epoch_range} step: Training\nElapsed time: {elapsed_time:.2f}s")
        # Training
        model_quali.train()
        for batch in train_dataloader:
            # Get data
            tokens, labels = batch
    
            # Forward pass
            outputs = model_quali(**tokens)
    
            # Compute loss (unsqueezing labels to add extra one gives error)
            loss = criterion(outputs.logits, labels)
    
            # Backward pass
            loss.backward()
    
            # Update weights
            optimizer.step()
    
            # Clear gradients
            optimizer.zero_grad()
    
        elapsed_time = time.time() - start_time
        print(f"-- on hyper epoch {hyper_epoch} and using random state {random_state[epoch]} --")
        print(f"Epoch {epoch + 1} of {epoch_range} step: Validation\nElapsed time: {elapsed_time:.2f}s")
        # Validation
        model_quali.eval()
        f1_scores = []
        with torch.no_grad():
            for batch in test_dataloader:
                # Get data
                tokens, labels = batch
                labels = (labels >= 0.5).float()  # convert probabilities to predictions
    
                # Forward pass
                outputs = model_quali(**tokens)
    
                # Compute probs
                probs = torch.sigmoid(outputs.logits)  # convert logits to probabilities
                preds = (probs >= 0.5).float()  # convert probabilities to predictions
    
                # Compute validation metrics here, e.g. F1 score
                # compare predictions with true labels
                f1_scores.append(f1_score(labels.numpy(), preds.numpy(), average='samples'))  # or 'micro', 'macro', 'weighted'
        # Calculate the average accuracy
        average_f1_score = np.mean(f1_scores)
        elapsed_time = time.time() - start_time
        print(f"Average F1 score for epoch {epoch + 1} of {epoch_range}: {average_f1_score * 100:.2f}%\nElapsed time: {elapsed_time:.2f}s")
        if average_f1_score + 0.08 > last_average_f1_score:
            epoch_broken = 0
            torch.save(model_quali.state_dict(), "../../model_quali.pt")
            last_average_f1_score = average_f1_score
        else:
            epoch_broken += 1
            if epoch_broken > 2: break
#'''

'\nlast_average_f1_score = 0\nepoch_range = 6\nstart_time = time.time()\nepoch_broken = 0\nrandom_state = [0] * epoch_range\nfor hyper_epoch in range(3):\n    if epoch_broken > 2: break\n    print(f\'______________________\nHyper epoch {hyper_epoch}\n______________________\n\')\n    for epoch in range(epoch_range):\n        elapsed_time = time.time() - start_time\n        if hyper_epoch == 0:\n            random.seed(elapsed_time)\n            # Generate a new random state for each epoch\n            random_state[epoch] = random.randint(0, 409)\n        # the dataframe will be sliced in epoch_range parts, each epoch will use 1/epoch_range of the dataframe\n        df_slice = df[df.index % epoch_range == epoch]\n        # X will have only columns 1, 2, 3 and 4, which are ementa, artigo_1, ocorrencia and ocorrencia_revogada\n        # y will have the row.iloc[13:] equivalent, which are the labels\n        X_train, X_test, y_train, y_test = train_test_split(df_slice.iloc[:, 1:5], df_slice

In [205]:
'''
# THIS IS TO TEST THE OLD MODEL, WE GOT 52.04% IN THE UPDATED MODEL
del model_quali
# load the model
model_quali_old = AutoModelForSequenceClassification.from_pretrained("../../model_quali_old", num_labels=81)

last_average_f1_score = 0
epoch_range = 6
start_time = time.time()
epoch_broken = 0
random_state = [0] * epoch_range
epoch = 4

elapsed_time = time.time() - start_time

# Generate a new random state for each epoch
random_state[epoch] = 226
# the dataframe will be sliced in epoch_range parts, each epoch will use 1/epoch_range of the dataframe
df_slice = df[df.index % epoch_range == epoch]
# X will have only columns 1, 2, 3 and 4, which are ementa, artigo_1, ocorrencia and ocorrencia_revogada
# y will have the row.iloc[13:] equivalent, which are the labels
X_train, X_test, y_train, y_test = train_test_split(df_slice.iloc[:, 1:5], df_slice.iloc[:, 13:], test_size=0.25, random_state=random_state[epoch])
# Recreate DataLoader for training and testing (validation)
train_dataset = LeiQualiDataset(pd.concat([X_train, y_train], axis=1), tokenizer)
test_dataset = LeiQualiDataset(pd.concat([X_test, y_test], axis=1), tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=7, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=7)

print(f"Epoch {epoch + 1} of {epoch_range} step: Validation\nElapsed time: {elapsed_time:.2f}s")
# Validation
model_quali_old.eval()
f1_scores = []
with torch.no_grad():
    for batch in test_dataloader:
        # Get data
        tokens, labels = batch
        labels = (labels >= 0.5).float()  # convert probabilities to predictions

        # Forward pass
        outputs = model_quali_old(**tokens)

        # Compute probs
        probs = torch.sigmoid(outputs.logits)  # convert logits to probabilities
        preds = (probs >= 0.5).float()  # convert probabilities to predictions

        # Compute validation metrics here, e.g. F1 score
        # compare predictions with true labels
        f1_scores.append(f1_score(labels.numpy(), preds.numpy(), average='samples'))  # or 'micro', 'macro', 'weighted'
# Calculate the average accuracy
average_f1_score = np.mean(f1_scores)
elapsed_time = time.time() - start_time
print(f"Average F1 score for epoch {epoch} of {epoch_range}: {average_f1_score * 100:.2f}%\nElapsed time: {elapsed_time:.2f}s")

del model_quali_old
#'''

'\n# THIS IS TO TEST THE OLD MODEL, WE GOT 52.04% IN THE UPDATED MODEL\ndel model_quali\n# load the model\nmodel_quali_old = AutoModelForSequenceClassification.from_pretrained("../../model_quali_old", num_labels=81)\n\nlast_average_f1_score = 0\nepoch_range = 6\nstart_time = time.time()\nepoch_broken = 0\nrandom_state = [0] * epoch_range\nepoch = 4\n\nelapsed_time = time.time() - start_time\n\n# Generate a new random state for each epoch\nrandom_state[epoch] = 226\n# the dataframe will be sliced in epoch_range parts, each epoch will use 1/epoch_range of the dataframe\ndf_slice = df[df.index % epoch_range == epoch]\n# X will have only columns 1, 2, 3 and 4, which are ementa, artigo_1, ocorrencia and ocorrencia_revogada\n# y will have the row.iloc[13:] equivalent, which are the labels\nX_train, X_test, y_train, y_test = train_test_split(df_slice.iloc[:, 1:5], df_slice.iloc[:, 13:], test_size=0.25, random_state=random_state[epoch])\n# Recreate DataLoader for training and testing (validati

# MODEL_QUALI.PT TRAINED! BETTER THAN THE OLD ONE, SOMEHOW, EVEN IF THE OLD ONE WAS RISKING OVERFITTING
#### Next step: use the model to predict the labels of the dataframe
##### After that we have to fit the predicted dataframe back to 247 rows, by taking the original_ids and the predicted labels and essembling them back together

In [206]:
# load the model
model_quali = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=81)
# Load the weights
model_quali.load_state_dict(torch.load("../../model_quali.pt"))

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

<All keys matched successfully>

In [207]:
# from df_all we do the opposite that we did to take the data for the training and testing
# keep only the rows for which the column #13 is NaN
df = df_all[df_all.iloc[:, 13].isna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2348 entries, 767 to 3382
Data columns (total 94 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   original_id                                  2348 non-null   int64  
 1   ementa                                       2348 non-null   object 
 2   artigo_1                                     2348 non-null   object 
 3   ocorrencia                                   2133 non-null   object 
 4   ocorrencia_revogada                          208 non-null    object 
 5   table_found                                  2348 non-null   int64  
 6   outro_assunto                                2348 non-null   float64
 7   n_palavras                                   2348 non-null   int64  
 8   freq_assunto                                 2348 non-null   float64
 9   artigo_1_revogado                            2348 non-null   int64  
 10

In [208]:
# normalizethe column 'n_palavras' in the following way df_norm = (df - df.min()) / (df.max() - df.min())
df['n_palavras'] = (df['n_palavras'] - df['n_palavras'].min()) / (df['n_palavras'].max() - df['n_palavras'].min())
display(df.n_palavras)

<ipython-input-208-fff8b4925071>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_palavras'] = (df['n_palavras'] - df['n_palavras'].min()) / (df['n_palavras'].max() - df['n_palavras'].min())


767     0.009286
768     0.009286
769     0.009286
770     0.009286
771     0.009286
          ...   
3378    1.000000
3379    1.000000
3380    0.000000
3381    0.017215
3382    0.017215
Name: n_palavras, Length: 2348, dtype: float64

In [209]:
'''
# # X will have only columns 1, 2, 3 and 4, which are ementa, artigo_1, ocorrencia and ocorrencia_revogada
# # y will have the row.iloc[13:] equivalent, which are the labels
# X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:5], df.iloc[:, 13:], test_size=0.25, random_state=409)
# # Recreate DataLoader for training and testing (validation)
# train_dataset = LeiQualiDataset(pd.concat([X_train, y_train], axis=1), tokenizer)
# test_dataset = LeiQualiDataset(pd.concat([X_test, y_test], axis=1), tokenizer)
# train_dataloader = DataLoader(train_dataset, batch_size=7, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=7)

# The code above was for training, now my df has NaN values in df.iloc[:, 13:], so I have to predict them with df.iloc[:, 1:5]

predict_dataset = LeiQualiDataset(pd.concat([df.iloc[:, 1:5], df.iloc[:, 13:]], axis=1), tokenizer)
predict_dataloader = DataLoader(predict_dataset, batch_size=7)

# Put the model in eval mode
model_quali.eval()

# Disable gradient calculations
with torch.no_grad():
    predictions = []
    for batch in predict_dataloader:
        inputs, _ = batch
        outputs = model_quali(**inputs)
        # Get the predicted probabilities
        predicted_probs = torch.sigmoid(outputs.logits)
        predictions.append(predicted_probs)

# Concatenate all batch predictions
predictions = torch.cat(predictions)

# Convert predicted probabilities to binary predictions (0 or 1) using a threshold, e.g., 0.5
threshold_a = 0.45
threshold_b = 0.55
ternary_predictions = torch.where(predictions < threshold_a, 0.0, torch.where(predictions > threshold_b, 1.0, 0.5))
len(ternary_predictions)
#'''

'\n# # X will have only columns 1, 2, 3 and 4, which are ementa, artigo_1, ocorrencia and ocorrencia_revogada\n# # y will have the row.iloc[13:] equivalent, which are the labels\n# X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:5], df.iloc[:, 13:], test_size=0.25, random_state=409)\n# # Recreate DataLoader for training and testing (validation)\n# train_dataset = LeiQualiDataset(pd.concat([X_train, y_train], axis=1), tokenizer)\n# test_dataset = LeiQualiDataset(pd.concat([X_test, y_test], axis=1), tokenizer)\n# train_dataloader = DataLoader(train_dataset, batch_size=7, shuffle=True)\n# test_dataloader = DataLoader(test_dataset, batch_size=7)\n\n# The code above was for training, now my df has NaN values in df.iloc[:, 13:], so I have to predict them with df.iloc[:, 1:5]\n\npredict_dataset = LeiQualiDataset(pd.concat([df.iloc[:, 1:5], df.iloc[:, 13:]], axis=1), tokenizer)\npredict_dataloader = DataLoader(predict_dataset, batch_size=7)\n\n# Put the model in eval mode\nmod

In [210]:
# ternary_predictions

In [211]:
# # Convert the tensor of predictions to a DataFrame
# ternary_predictions_df = pd.DataFrame(ternary_predictions.numpy(), columns=df.columns[13:], index=df.index)
# 
# # Replace the NaN values in df.iloc[:, 13:] with your predictions
# df.iloc[:, 13:] = df.iloc[:, 13:].fillna(value=ternary_predictions_df)
# df.info()

In [212]:
# save df in a csv file
# df.to_csv('../../df_ternary_predictions.csv', index=False)

Maybe you're wonder why am I saving and loading this file.
It is because it takes 20 minutes to run the commented code above, so I kept the results.
It was the same thing with the transformers model.

In [213]:
# load df from a csv file
df = pd.read_csv('../../df_ternary_predictions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348 entries, 0 to 2347
Data columns (total 94 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   original_id                                  2348 non-null   int64  
 1   ementa                                       2348 non-null   object 
 2   artigo_1                                     2348 non-null   object 
 3   ocorrencia                                   2133 non-null   object 
 4   ocorrencia_revogada                          208 non-null    object 
 5   table_found                                  2348 non-null   int64  
 6   outro_assunto                                2348 non-null   float64
 7   n_palavras                                   2348 non-null   float64
 8   freq_assunto                                 2348 non-null   float64
 9   artigo_1_revogado                            2348 non-null   int64  
 10  

# Fit back to 247 rows
### Now it is time
##### to fit the predicted dataframe back to 247 rows, by taking the original_ids and the predicted labels and essembling them back togethe
Actually, I will fit it back to the df_all that has all the occurrences first, then I essemble it back to the df_247.

In [214]:
df_all_backup = df_all.copy()

In [215]:
# pd.concat([df_all.head(10), df_all.tail(10)]), but add the ones with 61 and 62 original ids in the middle: 
pd.concat([df_all.iloc[:10, :], df_all.iloc[760:770, :], df_all.iloc[-10:, :]])

,original_id,ementa,artigo_1,ocorrencia,ocorrencia_revogada,table_found,outro_assunto,n_palavras,freq_assunto,artigo_1_revogado,...,ramo_sociologico_saúde,ramo_sociologico_trabalho,ramo_sociologico_ética,ramo_sociologico_rural,ramo_sociologico_cultura,ramo_sociologico_esporte,imposicao_reconhecimento_reconhece,imposicao_reconhecimento_impõe,ferramenta_ambiente_ambiente,ferramenta_ambiente_ferramenta
0,1,Altera o Título VI da Parte Especial do Decret...,Esta Lei altera o Título VI da Parte Especial ...,_o_ incorre nas penas previstas caput deste a...,NaN,0,1.0,1799,0.00111,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",esta lei estabelece princípios garantias dire...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem como funda...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem seguintes ...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",viii liberdade dos modelos negócios promovidos...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
5,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem por objeti...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
6,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",direito acesso internet todos,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
7,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",iii inovação fomento ampla difusão novas tecno...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
8,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",adesão padrões tecnológicos abertos que permi...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
9,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",internet sistema constituído conjunto protoco...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [216]:
# fit back to df_all
# all rows that match df_all[df_all.iloc[:, 13].isna()] will receive df.iloc[:, 13:] values in the same order as they appear in df_all
df_all.loc[df_all.iloc[:, 13].isna(), df.columns[13:]] = df.iloc[:, 13:].values

# pd.concat([df_all.head(10), df_all.tail(10)]), but add the ones with 61 and 62 original ids in the middle: 
pd.concat([df_all.iloc[:10, :], df_all.iloc[760:770, :], df_all.iloc[-10:, :]])

,original_id,ementa,artigo_1,ocorrencia,ocorrencia_revogada,table_found,outro_assunto,n_palavras,freq_assunto,artigo_1_revogado,...,ramo_sociologico_saúde,ramo_sociologico_trabalho,ramo_sociologico_ética,ramo_sociologico_rural,ramo_sociologico_cultura,ramo_sociologico_esporte,imposicao_reconhecimento_reconhece,imposicao_reconhecimento_impõe,ferramenta_ambiente_ambiente,ferramenta_ambiente_ferramenta
0,1,Altera o Título VI da Parte Especial do Decret...,Esta Lei altera o Título VI da Parte Especial ...,_o_ incorre nas penas previstas caput deste a...,NaN,0,1.0,1799,0.00111,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",esta lei estabelece princípios garantias dire...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem como funda...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem seguintes ...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",viii liberdade dos modelos negócios promovidos...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
5,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",disciplina uso internet brasil tem por objeti...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
6,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",direito acesso internet todos,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
7,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",iii inovação fomento ampla difusão novas tecno...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
8,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",adesão padrões tecnológicos abertos que permi...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
9,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",internet sistema constituído conjunto protoco...,NaN,0,0.0,6698,0.01911,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [217]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3383 entries, 0 to 3382
Data columns (total 94 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   original_id                                  3383 non-null   int64  
 1   ementa                                       3383 non-null   object 
 2   artigo_1                                     3383 non-null   object 
 3   ocorrencia                                   3095 non-null   object 
 4   ocorrencia_revogada                          276 non-null    object 
 5   table_found                                  3383 non-null   int64  
 6   outro_assunto                                3383 non-null   float64
 7   n_palavras                                   3383 non-null   int64  
 8   freq_assunto                                 3383 non-null   float64
 9   artigo_1_revogado                            3383 non-null   int64  
 10  

# Time to essemble it back to df (with the original number of rows, 247)

In [218]:
df = df_all

# Step 1: Drop the 'ocorrencia' and 'ocorrencia_revogada' columns
df.drop(['ocorrencia', 'ocorrencia_revogada'], axis=1, inplace=True)

# Step 2: Group the DataFrame by 'original_id' and compute the mean of the columns from 13 onwards
grouped_means = df.iloc[:, 13:].groupby(df['original_id']).mean()

# Step 3: Apply the ternary operation to the columns using the thresholds a and b
threshold_a = 0.25
threshold_b = 0.75
grouped_means = grouped_means.applymap(lambda x: 0.0 if x < threshold_a else 1.0 if x > threshold_b else 0.5)

# Step 4: For columns from 0 to 12, get the first value for each 'original_id'
first_values = df.iloc[:, :13].groupby(df['original_id']).first()

# Step 5: Concatenate the two DataFrames
df = pd.concat([first_values, grouped_means], axis=1)

display(df.head())

,original_id,ementa,artigo_1,table_found,outro_assunto,n_palavras,freq_assunto,artigo_1_revogado,internacional,f,...,ramo_sociologico_saúde,ramo_sociologico_trabalho,ramo_sociologico_ética,ramo_sociologico_rural,ramo_sociologico_cultura,ramo_sociologico_esporte,imposicao_reconhecimento_reconhece,imposicao_reconhecimento_impõe,ferramenta_ambiente_ambiente,ferramenta_ambiente_ferramenta
original_id,,,,,,,,,,,,,,,,,,,,,
1,1,Altera o Título VI da Parte Especial do Decret...,Esta Lei altera o Título VI da Parte Especial ...,0,1.0,1799,0.00111,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2,"Estabelece princípios, garantias, direitos e d...","Esta Lei estabelece princípios, garantias, dir...",0,0.0,6698,0.01911,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,3,Obriga o fornecedor de ingresso para evento cu...,O fornecedor de ingresso para evento cultural ...,0,0.0,328,0.00610,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,4,"Dispõe sobre propaganda, financiamento e prest...","A Lei nº 9.504, de 30 de setembro de 1997, pas...",0,1.0,1434,0.00279,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
5,5,Normatiza a divulgação de documentos instituci...,Esta Lei normatiza a divulgação de documentos ...,0,0.0,181,0.04972,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [219]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 1 to 247
Data columns (total 92 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   original_id                                  247 non-null    int64  
 1   ementa                                       247 non-null    object 
 2   artigo_1                                     247 non-null    object 
 3   table_found                                  247 non-null    int64  
 4   outro_assunto                                247 non-null    float64
 5   n_palavras                                   247 non-null    int64  
 6   freq_assunto                                 247 non-null    float64
 7   artigo_1_revogado                            247 non-null    int64  
 8   internacional                                247 non-null    float64
 9   f                                            247 non-null    float64
 10  fx

In [223]:
# rename original_id to id
df.rename(columns={'original_id': 'id'}, inplace=True)

# save df in a csv file
# df.to_csv('../../df_predicted.csv', index=False)

### Obs: table_found is all 0. Something went wrong here. Deleted on the google sheets and on the next cell.

In [224]:
df.drop(['table_found'], axis=1, inplace=True)

In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 1 to 247
Data columns (total 91 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   id                                           247 non-null    int64  
 1   ementa                                       247 non-null    object 
 2   artigo_1                                     247 non-null    object 
 3   outro_assunto                                247 non-null    float64
 4   n_palavras                                   247 non-null    int64  
 5   freq_assunto                                 247 non-null    float64
 6   artigo_1_revogado                            247 non-null    int64  
 7   internacional                                247 non-null    float64
 8   f                                            247 non-null    float64
 9   fx                                           247 non-null    float64
 10  di

______

______

______

______

In [220]:
'''
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
#'''

'\nfrom keras.models import Sequential\nfrom keras.layers import Dense\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import MultiLabelBinarizer\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom scipy.sparse import hstack\n#'

My code previously:
print("All occurrences that are current:\n" + "["+";".join(all_occ_curr[:2])+",...,"+";".join(all_occ_curr[-2:])+"]")
print("All occurrences that are not current:\n" + "["+";".join(all_occ_ncurr[:2])+";...;"+";".join(all_occ_ncurr[-2:])+"]")
My console:
All occurrences that are current:
[ disciplina uso internet brasil tem como fundamento respeito liberdade expressão bem como;disciplina uso internet brasil tem seguintes princípios;...; iniciativas públicas fomento cultura digital promoção internet como ferramenta social devem;promover inclusão digital]
All occurrences that are not current:
[vii aplicações internet conjunto funcionalidades que podem ser acessadas por meio terminal conectado internet redação dada pela medida provisória rejeitada;viii registros acesso aplicações internet conjunto informações referentes data hora uso uma determinada aplicação internet partir determinado endereço redação dada pela medida provisória rejeitada;...; utilização ensino uso aplicações internet sítios eletrônicos tecnologia informação com objetivo violar patente marca registrada direito autoral outros direitos propriedade intelectual incluído pela medida provisória rejeitada;poderá ocorrer por meio eletrônico acordo com regras uso rede social incluído pela medida provisória rejeitada]

I have a pandas DataFrame with 247 rows, and I have 247 text files (complete laws), what you have just saw was my lists all_occ_curr and all_occ_ncurr, it shows only the important fragments of the text files, so I avoid making the AI read the entire file for no reason. Now, these lists columns on my df, each row is a string separated by ";". My pandas df has some of it's columns completely filled by automation, that have some objective data such as word counts, subject match counts, mentions of other laws, which laws, etc... If it can be useful for machine learning, I'm all up to use those. On the other hand I have 61 rows of 9 columns filled by hand, these columns are (pt-br):
dimensão,benevolência,função social,função social especificamente ligada ao assunto,ramo (dogmático) do direito,ramo (sociológico) do direito,imposição/reconhecimento (pelo jurídico),ferramenta/ambiente.
My idea is to train a neural network with these 61 completed rows and be able to predict the values for these 9 columns for the other 186 rows. 61 values are the training set, they have the right answer to feedback after all.
I want to use keras to for deep learning, I will have 21 X columns and 9 y columns.

Based on your description, it seems like you want to apply a multi-label classification model, where each of your documents (laws) can have multiple labels (the 9 columns you mentioned).

You're correct that Keras can be used for this kind of task. The general steps would be as follows:

1. **Preprocess the text**: Convert the text into a format that can be used by a machine learning model. This could be a simple bag-of-words model or a more complex model like TF-IDF or word embeddings.

2. **Transform the labels**: The labels will need to be converted into a format that can be used by a machine learning model. For multi-label classification, this will typically be a binary matrix where each row corresponds to a document and each column corresponds to a label. If a document has a particular label, the corresponding entry in the matrix will be 1, otherwise it will be 0.

3. **Split the data into a training and validation set**: This will allow you to evaluate the performance of the model.

4. **Create the model**: This will be a neural network with an input layer that accepts the preprocessed text, one or more hidden layers, and an output layer with a number of units equal to the number of labels. Because this is a multi-label classification problem, the output layer will typically use a sigmoid activation function, which will allow each unit to output a value between 0 and 1 independently of the other units.

5. **Compile the model**: When you compile the model, you'll need to specify the loss function and the optimizer. For multi-label classification, the binary crossentropy loss is often used. The optimizer could be something like Adam or RMSprop.

6. **Train the model**: Fit the model to the training data for a certain number of epochs. You can also provide the validation data to monitor the validation loss and accuracy during training.

7. **Evaluate the model**: Use the model to make predictions on the validation data and evaluate the performance of the model.

8. **Predict on new data**: Once you're satisfied with your model's performance, you can use it to predict the labels of the 186 rows you haven't labeled yet.

Here is a simplified example:

```python
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess the text
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])

# Transform the labels
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['labels'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the model
model = Sequential()
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
model.add(Dense(y.shape[1], activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

# Predict on new data
new_data = vectorizer.transform(new_text)
predictions = model.predict(new_data)
labels = mlb.inverse_transform(predictions)
```

This is a basic example and might not work perfectly for your specific problem. You might need to adjust the preprocessing, the model architecture, the loss function, the optimizer, or other parts of the code. Additionally, deep learning models often require a lot of data to train effectively. If your dataset is small, you might want to consider using a simpler model like Naive Bayes or SVM, or you might want to use techniques like data augmentation to increase the size of your dataset.

In [221]:
# df['freq_assunto'] = df['freq_assunto'].str.replace(',', '.').str.replace('%', '').astype(float) / 100

In [222]:
'''
# Preprocess the text
vectorizer = TfidfVectorizer()

# Compute TF-IDF vectors for accepted text and rejected text separately
X_accepted = vectorizer.fit_transform(df['all_occ_curr'])
X_rejected = vectorizer.fit_transform(df['all_occ_ncurr'])

# Manually adjust the vectors
X_accepted *= 1.0  # weight for accepted text
X_rejected *= 0.3  # weight for rejected text

X_fs_accepted = vectorizer.fit_transform(df['quais_f'])
X_fxs_accepted = vectorizer.fit_transform(df['quais_fx'])
X_fs_rejected = vectorizer.fit_transform(df['quais_f_o'])
X_fxs_rejected = vectorizer.fit_transform(df['quais_fx_o'])

# Manually adjust the vectors
X_fs_accepted *= 1.0  # weight for accepted text
X_fxs_accepted *= 1.0  # weight for accepted text
X_fs_rejected *= 0.01  # weight for rejected text
X_fxs_rejected *= 0.01  # weight for rejected text

# X columns:
# data,assunto: emenda lei [f()] / é emendado [f(x)],outro assunto [o],assunto,n_palavras,freq_assunto,quais_f,quais_fx,quais_f_o,quais_fx_o,epígrafe,ementa,Artigo 1º,URL,Artigo 1 revogado,efetivamente revogada [r],table_found,all_occ_curr,all_occ_ncurr

# Combine the vectors
X = hstack([X_accepted, X_rejected,
            vectorizer.fit_transform(df['table_found'].fillna('zero').replace('f', 'UMum').astype(str)),
            vectorizer.fit_transform(df['assunto: emenda lei [f()] / é emendado [f(x)]'].fillna('zero').replace('f()', 'UMum').replace(';f()', 'UMum').replace('f(x)', 'dois').replace('f();f(x)', 'tres').astype(str)),
            vectorizer.fit_transform(df['outro assunto [o]'].fillna('zero').replace('o', 'UMum').replace('o;?', 'UMum').astype(str)),
            vectorizer.fit_transform(df['assunto'].astype(str)),
            vectorizer.fit_transform(df['freq_assunto'].astype(str)),
            X_accepted, X_fxs_accepted, X_fs_rejected, X_fxs_rejected,
            vectorizer.fit_transform(df['ementa']),
            vectorizer.fit_transform(df['Artigo 1º']),
            vectorizer.fit_transform(df['Artigo 1 revogado'].fillna('zero').replace('r', 'dois').replace('1/2r', 'UMum').astype(str)),
            vectorizer.fit_transform(df['efetivamente revogada [r]'].fillna('zero').replace('r', 'dois').replace('1/2r', 'UMum').astype(str))], format='csr')

# y columns:
# dimensão,benevolência,função social,função social especificamente ligada ao assunto,ramo (dogmático) do direito,ramo (sociológico) do direito,imposição/reconhecimento (pelo jurídico),ferramenta/ambiente

ys = [df['dimensão'],
      df['benevolência'],
      df['função social'],
      df['função social especificamente ligada ao assunto'],
      df['ramo (dogmático) do direito'],
      df['ramo (sociológico) do direito'],
      df['imposição/reconhecimento (pelo jurídico)'],
      df['ferramenta/ambiente']]

index_to_name = {
    0: 'dimensão',
    1: 'benevolência',
    2: 'função social',
    3: 'função social especificamente ligada ao assunto',
    4: 'ramo (dogmático) do direito',
    5: 'ramo (sociológico) do direito',
    6: 'imposição/reconhecimento (pelo jurídico)',
    7: 'ferramenta/ambiente'
}

for idx, o_y in enumerate(ys):
    X_labeled = X[:61]  # the first 61 rows (labeled data)
    y_labeled = y[:61]

    # Transform the labels
    mlb = MultiLabelBinarizer()
    y = mlb.fit_transform(o_y.astype(str).str.split(';').apply(lambda x: [i.strip() for i in x if i.strip() != '']))

    # Split the labeled data into a training set and a validation set
    X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.3, random_state=42)

    # Create the model
    model = Sequential()
    model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(y.shape[1], activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam')

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Loss[{idx}]:', loss)
    print(f'Accuracy[{idx}]:', accuracy)

    # Predict on new data
    for jdx, _ in enumerate(X[61:]):
        new_data = X[61+jdx:61+jdx+1]
        predictions = model.predict(new_data)
        df.at[61+jdx, index_to_name[idx]] = mlb.inverse_transform(predictions)[0]
#'''

"\n# Preprocess the text\nvectorizer = TfidfVectorizer()\n\n# Compute TF-IDF vectors for accepted text and rejected text separately\nX_accepted = vectorizer.fit_transform(df['all_occ_curr'])\nX_rejected = vectorizer.fit_transform(df['all_occ_ncurr'])\n\n# Manually adjust the vectors\nX_accepted *= 1.0  # weight for accepted text\nX_rejected *= 0.3  # weight for rejected text\n\nX_fs_accepted = vectorizer.fit_transform(df['quais_f'])\nX_fxs_accepted = vectorizer.fit_transform(df['quais_fx'])\nX_fs_rejected = vectorizer.fit_transform(df['quais_f_o'])\nX_fxs_rejected = vectorizer.fit_transform(df['quais_fx_o'])\n\n# Manually adjust the vectors\nX_fs_accepted *= 1.0  # weight for accepted text\nX_fxs_accepted *= 1.0  # weight for accepted text\nX_fs_rejected *= 0.01  # weight for rejected text\nX_fxs_rejected *= 0.01  # weight for rejected text\n\n# X columns:\n# data,assunto: emenda lei [f()] / é emendado [f(x)],outro assunto [o],assunto,n_palavras,freq_assunto,quais_f,quais_fx,quais_f_o,